Import Keras

In [ ]:
library(keras)

Create the Data Generators

In [ ]:
train_data_gen <- image_data_generator(
      preprocessing_function = nasnet_preprocess_input, #split into 80% training and 20% validation data
      rotation_range = 40, #Data Augmentation
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = TRUE,
      fill_mode = "nearest"
)

# Validation data shouldn't be augmented! But it should also be scaled.
valid_data_gen <- image_data_generator(
      preprocessing_function = nasnet_preprocess_input
)
   

Specify the Params

In [ ]:
img_width <- 331L
img_height <- 331L
channels <- 3L #RGB has 3 colours

batch_size <- 32L

num_classes <- 3L

train_image_files_path <- '../input/testing/Train'

Load the Data and convert IMG to arrays

In [ ]:
# training images
train_it <- flow_images_from_directory(train_image_files_path, 
                                       train_data_gen,
                                       target_size = c(img_width, img_height),
                                       class_mode = "categorical",
                                       batch_size = batch_size,
                                       shuffle = TRUE) 

# validation images
val_it <- flow_images_from_directory(train_image_files_path, 
                                     valid_data_gen,
                                     target_size = c(img_width, img_height),
                                     class_mode = "categorical",
                                     batch_size = batch_size,
                                     shuffle = FALSE)

Check if loading worked

In [ ]:
str(reticulate::iter_next(train_it))
str(reticulate::iter_next(val_it))

Create the Model

In [ ]:
# Load the pretrained Model (Transfer Learning)
base_model <- application_nasnetlarge(weights = 'imagenet', include_top = FALSE, 
                                input_shape = c(img_width, img_height, channels)) # We stripped off the input layer and the classifier layer

# Examine the Architecture
summary(base_model)

Fine Tuning

In [ ]:
# define new classifier layers
predictions <- base_model$output %>%
  layer_global_average_pooling_2d(trainable = T) %>%
  layer_dense(num_classes, trainable=T) %>%   
  layer_activation("softmax", trainable=T) # Softmax is used for Multi-label classification

# add the layers to the model
model <- keras_model(inputs = base_model$input, outputs = predictions)
freeze_weights(base_model) # Freeze the weights of the base_model as we only want to train the classifier

summary(model)

Compile the Model

In [ ]:
model %>% compile(
  optimizer = optimizer_sgd(lr = 0.001, momentum = 0.9, nesterov = TRUE),
  loss = 'categorical_crossentropy', # used in multi-label classification
  metrics = c('accuracy')
)

Start Training

In [ ]:
hist <- model %>% fit_generator(
  train_it,
  validation_data = val_it,
  epochs=10, 
  steps_per_epoch = 1,
  validation_steps = 1,
  verbose = 2
)

Save the Model

In [ ]:
save_model_hdf5(model, "model.h5")

Load Model

In [ ]:
model <- load_model_hdf5("model.h5")

Visualise the Training History

In [ ]:
plot(hist)

In [ ]:
z <- evaluate_generator(model, 
                        val_it, 
                        steps = 1)
z

Load the Test Image

In [ ]:
# load the image
img_path <- "../input/testing/Train/1/download1.jpeg"
img <- image_load(img_path, target_size = c(img_width,img_height))
x <- image_to_array(img)

Preprocess the IMG

In [ ]:
# ensure we have a 4d tensor with single element in the batch dimension,
# the preprocess the input for prediction using resnet50
x <- array_reshape(x, c(1, dim(x)))
x <- nasnet_preprocess_input(x)
x

Prediction

In [ ]:
# make predictions then decode and print them
pred <- model %>% predict(x)

In [ ]:
pred

In [ ]:
# Implement a dictionary (Key-Value) for mapping prediction to labels

d <- list()

d[[1]] <- "Apples"

d[[2]] <- "Oranges"

d[[3]] <- "Bananas"

In [ ]:
d[[which.max(pred)]]

In [ ]:
# test_image_files_path <- '../input/testing-2/Test'

# test_data_gen <- image_data_generator(
#       preprocessing_function = nasnet_preprocess_input
# ) 


# test_it <- flow_images_from_directory(test_image_files_path, 
#                                      test_data_gen,
#                                      target_size = c(img_width, img_height),
#                                      class_mode = NULL,
#                                      batch_size = batch_size,
#                                      shuffle = FALSE)

# str(reticulate::iter_next(test_it))

TESTING

In [ ]:
#' In this example we fine tune Mobile Net to better predict cats and
#' dogs in photos. It also demonstrates the usage of image data generators
#' for efficient preprocessing and training.
#' 
#' It's preferable to run this example in a GPU.

# Download data -----------------------------------------------------------

download.file(
  "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip", 
  destfile = "cats-dogs.zip"
)

# Pre-processing ----------------------------------------------------------

zip::unzip("cats-dogs.zip", exdir = "data-raw")

# We will organize images in the following structure:
# data/
#     train/
#          Cat/
#          Dog/
#     validation
#          Cat/
#          Dog/
#     test/
#          images/
#

all_imgs <- fs::dir_ls(
  "data-raw/PetImages/", 
  recursive = TRUE, 
  type = "file",
  glob = "*.jpg"
)

# some images are corrupt and we exclude them
# this will make sure all images can be read.
for (im in all_imgs) {
  out <- try(magick::image_read(im), silent = TRUE)
  if (inherits(out, "try-error")) {
    fs::file_delete(im)
    message("removed image: ", im)
  }
}

# re-list all imgs
all_imgs <- fs::dir_ls(
  "data-raw/PetImages/", 
  recursive = TRUE, 
  type = "file",
  glob = "*.jpg"
)

set.seed(5)

training_imgs <- sample(all_imgs, size = length(all_imgs)/2)
validation_imgs <- sample(all_imgs[!all_imgs %in% training_imgs], size = length(all_imgs)/4)         
testing_imgs <- all_imgs[!all_imgs %in% c(training_imgs, validation_imgs)]

# create directory structure
fs::dir_create(c(
  "data/train/Cat",
  "data/train/Dog",
  "data/validation/Cat",
  "data/validation/Dog",
  "data/test/images"
))

# copy training images
fs::file_copy(
  path = training_imgs, 
  new_path = gsub("data-raw/PetImages", "data/train", training_imgs)
)

# copy valid images
fs::file_copy(
  path = validation_imgs, 
  new_path = gsub("data-raw/PetImages", "data/validation", validation_imgs)
)

# copy testing imgs
fs::file_copy(
  path = testing_imgs,
  new_path = gsub("data-raw/PetImages/(Dog|Cat)/", "data/test/images/\\1", testing_imgs)
)

# Image flow --------------------------------------------------------------

library(keras)

training_image_gen <- image_data_generator(
  rotation_range = 20,
  width_shift_range = 0.2,
  height_shift_range = 0.2,
  horizontal_flip = TRUE,
  preprocessing_function = imagenet_preprocess_input
)

validation_image_gen <- image_data_generator(
  preprocessing_function = imagenet_preprocess_input
)

training_image_flow <- flow_images_from_directory(
  directory = "data/train/", 
  generator = training_image_gen, 
  class_mode = "binary",
  batch_size = 100,
  target_size = c(224, 224), 
)

validation_image_flow <- flow_images_from_directory(
  directory = "data/validation/", 
  generator = validation_image_gen, 
  class_mode = "binary",
  batch_size = 100,
  target_size = c(224, 224), 
  shuffle = FALSE
)

# Model -------------------------------------------------------------------

mob <- application_mobilenet(include_top = FALSE, pooling = "avg")
freeze_weights(mob)

model <- keras_model_sequential() %>% 
  mob() %>% 
  layer_dense(256, activation = "relu") %>% 
  layer_dropout(rate = 0.2) %>% 
  layer_dense(units = 1, activation = "sigmoid")

model %>% 
  compile(loss = "binary_crossentropy", optimizer = "adam", metrics = "accuracy")

model %>% fit_generator(
  generator = training_image_flow, 
  epochs = 1, 
  steps_per_epoch = training_image_flow$n/training_image_flow$batch_size,
  validation_data = validation_image_flow,
  validation_steps = validation_image_flow$n/validation_image_flow$batch_size
)

# now top layers weights are fine, we can unfreeze the lower layer weights.
unfreeze_weights(mob)

model %>% 
  compile(loss = "binary_crossentropy", optimizer = "adam", metrics = "accuracy")

model %>% fit_generator(
  generator = training_image_flow, 
  epochs = 3, 
  steps_per_epoch = training_image_flow$n/training_image_flow$batch_size,
  validation_data = validation_image_flow,
  validation_steps = validation_image_flow$n/validation_image_flow$batch_size
)

# Generate predictions for test data --------------------------------------

test_flow <- flow_images_from_directory(
  generator = validation_image_gen,
  directory = "data/test", 
  target_size = c(224, 224),
  class_mode = NULL,
  shuffle = FALSE
)

predictions <- predict_generator(
  model, 
  test_flow,
  steps = test_flow$n/test_flow$batch_size
)

magick::image_read(testing_imgs[1])
predictions[1]

magick::image_read(testing_imgs[6250])
predictions[6250]
